# Categorical Boost

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
pd.set_option("display.max_columns", 400)
pd.set_option("display.max_rows", 400)

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv", index_col='id')
test_df = pd.read_csv("../../data/Processed/test2.csv", index_col="id")
print(train_df.shape)
print(test_df.shape)

(132250, 191)
(28340, 190)


In [3]:
#train_df.head()

In [4]:
#train_df.info()

In [5]:
#test_df.head()

### 武器の情報を個人ごとからチームごとに変える
category1, 2を落とす、

他のカテゴリをチームごとにする

reskinそのまま

In [6]:
# 落とさないほうが精度上がってしまった。。。

"""suffixes = ["-A1", "-A2", "-A3", "-A4", "-B1", "-B2", "-B3", "-B4"]
drop_cols = ["category1", "category2"]

for c in drop_cols:
    for s in suffixes:
        col = c+s
        train_df.drop(col, axis=1, inplace=True)
        test_df.drop(col, axis=1, inplace=True)
"""

'suffixes = ["-A1", "-A2", "-A3", "-A4", "-B1", "-B2", "-B3", "-B4"]\ndrop_cols = ["category1", "category2"]\n\nfor c in drop_cols:\n    for s in suffixes:\n        col = c+s\n        train_df.drop(col, axis=1, inplace=True)\n        test_df.drop(col, axis=1, inplace=True)\n'

In [7]:
#train_df.columns

In [8]:
y = train_df["y"]
train_df = train_df.drop("y", axis=1)

In [9]:
# add team info
train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2"])

(160590, 190)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
(160590, 278)
complete


In [10]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [11]:
# add info about numeric column
from Functions import prepro

num_cols = ["level", "range", "rapid", "atack"]

train_df = prepro.flat(train_df, num_cols)
test_df = prepro.flat(test_df, num_cols)

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

In [12]:
# make input

drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]
X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode = False, verbose=False)

In [13]:
"""print(train_df.shape)
print(test_df.shape)
for col in X.columns:
    print(col)"""

'print(train_df.shape)\nprint(test_df.shape)\nfor col in X.columns:\n    print(col)'

In [14]:
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import Pool, CatBoostClassifier

categorical_features_indices = np.where(X.dtypes != np.float)[0]

random.seed(0)

In [15]:
# 全データを5つに分割
SIZE = X.shape[0]
K = 5

def make_kfolds(SIZE, K):
    # return list object, each element is indices of its fold
    FOLD_SIZE = int(SIZE/K)
    res = []
    indices = [i for i in range(SIZE)]
    for i in range(K-1):
        fold = random.sample(indices, FOLD_SIZE)
        indices = list(set(indices) - set(fold))
        res.append(fold)
    res.append(indices)
    return res

folds = make_kfolds(SIZE, K)
print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    

    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")


5
fold  1  size is  26450
fold  2  size is  26450
fold  3  size is  26450
fold  4  size is  26450
fold  5  size is  26450
successfully split


In [16]:
params = {
    "loss_function" : "Logloss",
    "eval_metric" : "Logloss",
    "iterations":1000,
    "learning_rate" : 0.1,
    "use_best_model": True,

    

}

THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []

all_indices = sum(folds, [])
for i in range(K):
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    
    train_X = X.iloc[train_indices]
    train_y = y.iloc[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y.iloc[valid_indices]
    
    train_data = Pool(train_X, train_y, cat_features=categorical_features_indices)
    valid_data = Pool(valid_X, valid_y, cat_features=categorical_features_indices)
    
    model = CatBoostClassifier(**params)

    
    model.fit(
        train_data,
        eval_set=valid_data,
        early_stopping_rounds=10,
        verbose=10,
        use_best_model=True,
    )
    
    pred = model.predict(valid_X)
    # pred = np.where(pred < THRESHOLD, 0, 1)
    train_pred.append(pred)
    temp += np.sum(pred)
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

0:	learn: 0.6924081	test: 0.6923785	best: 0.6923785 (0)	total: 841ms	remaining: 14m
10:	learn: 0.6882179	test: 0.6886489	best: 0.6886489 (10)	total: 7.57s	remaining: 11m 20s
20:	learn: 0.6857641	test: 0.6866889	best: 0.6866889 (20)	total: 14.2s	remaining: 11m 3s
30:	learn: 0.6842917	test: 0.6858829	best: 0.6858829 (30)	total: 21s	remaining: 10m 56s
40:	learn: 0.6829694	test: 0.6851030	best: 0.6851030 (40)	total: 27.8s	remaining: 10m 50s
50:	learn: 0.6816852	test: 0.6843217	best: 0.6843217 (50)	total: 34.6s	remaining: 10m 43s
60:	learn: 0.6806835	test: 0.6838233	best: 0.6838233 (60)	total: 41.7s	remaining: 10m 41s
70:	learn: 0.6798990	test: 0.6834917	best: 0.6834917 (70)	total: 48.5s	remaining: 10m 35s
80:	learn: 0.6789599	test: 0.6831008	best: 0.6831008 (80)	total: 55.9s	remaining: 10m 34s
90:	learn: 0.6781040	test: 0.6826937	best: 0.6826937 (90)	total: 1m 2s	remaining: 10m 27s
100:	learn: 0.6774139	test: 0.6824044	best: 0.6824044 (100)	total: 1m 10s	remaining: 10m 26s
110:	learn: 0.67

30:	learn: 0.6846017	test: 0.6862283	best: 0.6862283 (30)	total: 25.6s	remaining: 13m 21s
40:	learn: 0.6833089	test: 0.6855428	best: 0.6855428 (40)	total: 34.3s	remaining: 13m 22s
50:	learn: 0.6821480	test: 0.6849536	best: 0.6849536 (50)	total: 43.2s	remaining: 13m 24s
60:	learn: 0.6810895	test: 0.6843677	best: 0.6843677 (60)	total: 52.6s	remaining: 13m 30s
70:	learn: 0.6802770	test: 0.6840565	best: 0.6840565 (70)	total: 1m 1s	remaining: 13m 24s
80:	learn: 0.6793577	test: 0.6836893	best: 0.6836893 (80)	total: 1m 10s	remaining: 13m 24s
90:	learn: 0.6786041	test: 0.6835593	best: 0.6835593 (90)	total: 1m 19s	remaining: 13m 18s
100:	learn: 0.6776443	test: 0.6831122	best: 0.6831122 (100)	total: 1m 28s	remaining: 13m 6s
110:	learn: 0.6767453	test: 0.6828394	best: 0.6828390 (109)	total: 1m 37s	remaining: 12m 58s
120:	learn: 0.6758739	test: 0.6826278	best: 0.6826278 (120)	total: 1m 46s	remaining: 12m 50s
130:	learn: 0.6748665	test: 0.6824981	best: 0.6824927 (128)	total: 1m 55s	remaining: 12m 4

In [17]:
preds = []
for i in range(K):
    model = models[i]
    pred = model.predict(test_X)
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":pred})
# temp.hist(bins=20)

0.5755822159491885
0.5708892025405787
0.5848270995059985
0.5855328158080452
0.5782286520818631
0.5790119971771348


In [18]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_cat_2.csv')

0.5824629498941426


In [19]:
importance = pd.DataFrame(models[1].feature_importance(), index=X.columns, columns=['importance']).sort_values('importance')
display(importance)

AttributeError: 'CatBoostClassifier' object has no attribute 'feature_importance'

In [ ]:
submit_df